### 3차원 좌표 변환 정리

 - 먼저 euler angle coordinate를 transformation coordinate로 바꾸기
 - 그 다음은 python에서 transformation matrix를 다루기 위해 list type으로 저장하는 방법을 봄
 - 저장된 list type을 numpy를 사용하여 numpy.array로 바꾸기
 - 이렇게 데이터 타입이 바뀐 transformation matrix를 numpy에서 지원하는 matrix 연산을 해보기
 
 > 추후 pandas도 같이 사용하여 수기 작성 없이 변환 및 계산이 가능하게 함수 생성 필요!

#### euler coordinate에서 transformation matrix로 변환하기

 - 변환은 언제나 error 누적이라는 부분을 늘 가지고 있음을 알아야 함
 - 먼저 euler coordinate에서 rotation term 부분만 연산을 진행
 - 그 이후, translation term에 해당하는 부분을 추가하는 방식을 사용
 - 아래에 작성한 함수는 data라는 변수를 받는 데 이는 euler angle의 6 term을 가지는 list 
 - 즉, 0~5의 list index에 각각 x, y, z, Rx, Ry, Rz가 입력됨
 - return되는 값은 transformation matrix로 numpy.array 형식
 
 > 참고자료 : 
 > 
 > https://en.wikipedia.org/wiki/Rotation_matrix
 > 
 > http://easyspin.org/easyspin/documentation/eulerangles.html
 > 
 > https://www.learnopencv.com/rotation-matrix-to-euler-angles/

In [1]:
import numpy as np
import math

def euler2mtx(data):
    posX = data[0]
    posY = data[1]
    posZ = data[2]
    pitch = data[3]
    roll = data[4]
    yaw = data[5]
    
    Rx = np.array([[1, 0, 0, 0],
                   [0, math.cos(pitch), -1 * math.sin(pitch), 0],
                   [0, math.sin(pitch), math.cos(pitch), 0],
                   [0, 0, 0, 1]])
    
    Ry = np.array([[math.cos(roll), 0, math.sin(roll), 0],
                   [0, 1, 0, 0],
                   [-1 * math.sin(roll), 0, math.cos(roll), 0],
                   [0, 0, 0, 1]])
    
    Rz = np.array([[math.cos(yaw), -1 * math.sin(yaw), 0, 0],
                   [math.sin(yaw), math.cos(yaw), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    
    mtxR = np.dot(Rz, np.dot(Ry, Rx))
    
    mtxR[0][3] = posX
    mtxR[1][3] = posY
    mtxR[2][3] = posZ
    
    return mtxR

In [2]:
# test
euler_raw = np.array([602.156206084942,-180.355737056239,278.247962852899,-137.943489136756,-9.85814556786191,-0.00693582798166002])

mtx_result = euler2mtx(euler_raw)

print(mtx_result)

[[-9.07534912e-01  1.25355982e-01  4.00831962e-01  6.02156206e+02]
 [ 6.29460697e-03  9.58367683e-01 -2.85467621e-01 -1.80355737e+02]
 [-4.19929472e-01 -2.56548753e-01 -8.70541197e-01  2.78247963e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


#### transformation matrix에서 euler coordinate로 변환하기

 - 변환은 언제나 error 누적이라는 부분을 늘 가지고 있음을 알아야 함 (강조)
 - 위의 변환과 반대로 transformation matrix를 euler coordinate 다시 변환
 
 > 참고자료 :
 > 
 > https://www.learnopencv.com/rotation-matrix-to-euler-angles/

In [3]:
def mtx2euler(mtx) :
    dLocSqrt = (mtx[0][0] * mtx[0][0] +  mtx[1][0] * mtx[1][0])
    
    if(dLocSqrt<0.0):
        dLocSqrt = 0.0
    else:
        dLocSqrt= math.sqrt(dLocSqrt)
    
    dLocRy = math.atan2(-mtx[2][0], dLocSqrt)
    dLocCy = math.cos(dLocRy)
 
    if (dLocCy < 0.001745328365898):
        Rx = 0.0
        Ry = dLocRy
        Rz = math.atan2(-mtx[0][1], mtx[1][1])
    else:
        Rx = math.atan2(mtx[2][1]/dLocCy, mtx[2][2]/dLocCy)
        Ry = dLocRy
        Rz = math.atan2(mtx[1][0]/dLocCy, mtx[0][0]/dLocCy)
        
    X = mtx[0][3]
    Y = mtx[1][3]
    Z = mtx[2][3]
    Rx = Rx * 180 / 3.141592653589793238462643383279
    Ry = Ry * 180 / 3.141592653589793238462643383279
    Rz = Rz * 180 / 3.141592653589793238462643383279
 
    return np.array([X, Y, Z, Rx, Ry, Rz])

In [4]:
# test
mtx_raw = np.array([[0.985234673141268, 0.114597023338963, 0.127201439702577, 602.156206084942],
                    [-0.000119265647590905, -0.742498393044716, 0.669847870155848, -180.355737056239],
                    [0.171209438466156, -0.659972512525325, -0.731521493031881, 278.247962852899],
                    [0, 0, 0, 1]])

euler_result = mtx2euler(mtx_raw)

print(euler_result)

[ 6.02156206e+02 -1.80355737e+02  2.78247963e+02 -1.37943489e+02
 -9.85814557e+00 -6.93582798e-03]


#### 3차원 transformation 연산

 - 먼저, csv의 transformation data를 받았다고 가정 --> 여기서는 이를 수기로 list type에 저장
 - transformation matrix가 저장되어 있는 list type을 python에서 matrix 연산을 쉽게하기 위해 numpy.array로 바꿈
 - numpy.array로 변경된 데이터를 np.matmul을 사용하여 matrix multiplication을 진행
 > 참고자료 :
 >
 > https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html
 >
 > https://www.python-course.eu/matrix_arithmetic.php

In [5]:
def mulmulmtx(robot2base, ots2base, ots2marker, marker2bone, bone2imp):
    temp0 = np.dot(robot2base, np.linalg.inv(ots2base))
    temp1 = np.dot(temp0, ots2marker)
    temp2 = np.dot(temp1, marker2bone)
    temp3 = np.dot(temp2, bone2imp)
    
    result = temp3
    
    return result

In [6]:
robot2base = [ [-0.00378376225785651,0.236726351680864,0.9715689978396,205.722290937229],
               [-0.018341466823077,0.971396084105608,-0.236755651208307,397.849062498915],
               [-0.999824621490166,-0.0187158276375814,0.000666376685034192,325.234562908007],
               [0,0,0,1] ]

ots2base = [-94.0386558532715, 416.18765411377, -1899.82280883789, 0.159738227699664, 0.21711069767038, 0.00495654789126103]

ots2marker = [192.396050262451, 64.6055419921875, -1437.20073242188, -0.14098226825726, 0.0257808341326867, -0.853490130976349]

marker2bone = [ [-0.055625956505537,0.997107863426209,-0.0517850890755653,84.2919180746216],
                [-0.0610264055430889,0.0483730845153332,0.996963322162628,-200.645559860843],
                [0.996584951877594,0.0586172938346863,0.0581591054797173,-71.8515067853073],
                [0,0,0,1] ]

bone2imp = [ [1.000000, 0.000000, -0.000000, 0.499977],
             [-0.000000, 1.000000, 0.000000, -27.695404], 
             [0.000000, -0.000000, 1.000000, -16.773041], 
             [0.000000, 0.000000, 0.000000, 1.000000] ]

In [7]:
robot2base_mtx = np.array(robot2base)

ots2base_mtx = euler2mtx(ots2base)

ots2marker_mtx = euler2mtx(ots2marker)

marker2bone_mtx = np.array(marker2bone)

bone2imp_mtx = np.array(bone2imp)

In [8]:
usrcrd_mtx = mulmulmtx(robot2base_mtx, ots2base_mtx, ots2marker_mtx, marker2bone_mtx, bone2imp_mtx)

In [9]:
usrcrd = mtx2euler(usrcrd_mtx)

In [10]:
usrcrd

array([ 6.02156210e+02, -1.80355737e+02,  2.78247973e+02, -1.37943489e+02,
       -9.85814540e+00, -6.93572028e-03])

In [11]:
usrcrd_mtx

array([[ 9.85234668e-01,  1.14597027e-01,  1.27201435e-01,
         6.02156210e+02],
       [-1.19263795e-04, -7.42498389e-01,  6.69847867e-01,
        -1.80355737e+02],
       [ 1.71209435e-01, -6.59972497e-01, -7.31521478e-01,
         2.78247973e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])